In [0]:
# ==========================================================
# UNIFICAR 1FL HISTORICO
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================

from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F
import traceback
import bootstrap
from util.utils_liga1 import *

dbutils.widgets.text("filesystem", "")
dbutils.widgets.text("capa_raw", "")
dbutils.widgets.text("rutaBase", "")
dbutils.widgets.text("nombre_archivo", "")
dbutils.widgets.text("historical_start_year", "")
dbutils.widgets.text("current_year", "")

filesystem = dbutils.widgets.get("filesystem")
capa_raw = dbutils.widgets.get("capa_raw").strip("/")
rutaBase = dbutils.widgets.get("rutaBase").strip("/")
nombre_archivo = dbutils.widgets.get("nombre_archivo")
historical_start_year = int(dbutils.widgets.get("historical_start_year"))
current_year = int(dbutils.widgets.get("current_year"))

spark = SparkSession.builder.getOrCreate()

setup_adls()
dbutils = get_dbutils()
base_path = get_abfss_path(f"{capa_raw}/{rutaBase}/{nombre_archivo}")

print("===============================================")
print("UNIFICADOR HISTÓRICO DE PARQUETS (1FL)")
print("===============================================")
print(f"Entidad        : {nombre_archivo}")
print(f"Años procesados: {historical_start_year} - {current_year}")
print(f"Ruta base RAW  : {base_path}")
print("===============================================")



In [0]:
rutas_temp = []
for year in range(historical_start_year, current_year):
    ruta = f"{base_path}/temp/{year}/data"
    try:
        archivos = dbutils.fs.ls(ruta)
        if any(f.name.endswith(".parquet") for f in archivos):
            rutas_temp.append(ruta)
    except Exception:
        print(f"Carpeta no encontrada o vacía: {ruta}")

if not rutas_temp:
    raise Exception("No se encontraron archivos temporales para unificar.")

print(f"Carpetas detectadas: {len(rutas_temp)}")
for r in rutas_temp:
    print(f" - {r}")

df_final = None
columnas_ref = set()

for ruta in rutas_temp:
    try:
        df = spark.read.parquet(ruta)
        print(f"Leído correctamente: {ruta}")

        # Convertir campos complejos a string JSON para compatibilidad
        for c in df.columns:
            tipo = dict(df.dtypes).get(c, "")
            if tipo.startswith("array") or tipo.startswith("struct"):
                df = df.withColumn(c, F.to_json(c))

        columnas_actuales = set(df.columns)
        columnas_ref |= columnas_actuales

        if df_final is not None:
            faltantes_actual = list(columnas_ref - columnas_actuales)
            if faltantes_actual:
                df = df.selectExpr(*df.columns, *[f"NULL as {c}" for c in faltantes_actual])

            faltantes_final = list(columnas_ref - set(df_final.columns))
            if faltantes_final:
                df_final = df_final.selectExpr(*df_final.columns, *[f"NULL as {c}" for c in faltantes_final])

            df_final = df_final.unionByName(df, allowMissingColumns=True)
        else:
            df_final = df

    except AnalysisException as e:
        print(f"Error leyendo {ruta}: {str(e)}")
    except Exception:
        print(f"Error general en {ruta}: {traceback.format_exc()}")

if df_final is None:
    raise Exception("No se pudo generar DataFrame consolidado.")

total_filas = df_final.count()
print(f"Total filas unificadas: {total_filas}")

output_path = f"{base_path}/1FL/data"

print("===============================================")
print(f"Guardando consolidado final en carpeta: {output_path}")
print("===============================================")

df_final.write.mode("overwrite").parquet(output_path)

print(f"Consolidado guardado con éxito ({total_filas} registros).")
print("Proceso de unificación histórica finalizado con éxito.")
